<a href="https://colab.research.google.com/github/Danielp8s/DLP2/blob/main/Op2_DL2_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PARTE 1
# Importar las bibliotecas necesarias
import os                                             # Operaciones del sistema (rutas de archivos, directorios)
import torch                                          # Pytorch: Biblioteca principal para crear y entrenar redes neuronales
import torchvision                                    # Paquete para trabajar con imágenes y modelos preentrenados
from torchvision import datasets, transforms, models  # Para gestionar datasets y realizar transformaciones en imágenes
from torch.utils.data import DataLoader, Subset       # Para gestionar lotes de datos (dataloaders) y subconjuntos
import torch.nn as nn                                 # Para definir la estructura de la red neuronal (capas)
import torch.optim as optim                           # Para implementar algoritmos de optimización (Adam, SGD)
import matplotlib.pyplot as plt                       # Para graficar los resultados
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y validación
from sklearn.metrics import classification_report, confusion_matrix  # Para medir el rendimiento del modelo
import numpy as np                                    # Para cálculos y manipulación de arrays

# Verificar disponibilidad de GPU
# Si se dispone de una GPU compatible con CUDA, se utilizará; de lo contrario, se usará la CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#PARTE 2
# Montar Google Drive para acceder a los datos (solo en Google Colab)
from google.colab import drive
drive.mount('/content/drive')

import os  # Para manejar rutas de archivos y directorios

# Definir la ruta donde están los datos (carpeta "dogs_vs_cats" en Google Drive)
data_dir = '/content/drive/MyDrive/dogs_vs_cats'

# TRANSFORMACIONES PARA LOS DATOS
# Transformación de entrenamiento para VGG16 y ResNet50 (tamaño de entrada 128x128)
train_transforms_small = transforms.Compose([
    transforms.RandomResizedCrop(128),               # Recorte aleatorio y redimensionado a 128x128
    transforms.RandomHorizontalFlip(),               # Volteo horizontal aleatorio
    transforms.ToTensor(),                           # Convertir la imagen a tensor
    transforms.Normalize([0.485, 0.456, 0.406],      # Normalización con valores medios y desviaciones estándar (ImageNet)
                         [0.229, 0.224, 0.225])
])

# Transformación de prueba para VGG16 y ResNet50 (128x128)
test_transforms_small = transforms.Compose([
    transforms.Resize(150),                          # Redimensionar la imagen a 150px
    transforms.CenterCrop(128),                      # Recorte centrado a 128x128
    transforms.ToTensor(),                           # Convertir a tensor
    transforms.Normalize([0.485, 0.456, 0.406],      # Normalización (valores de ImageNet)
                         [0.229, 0.224, 0.225])
])

# Transformación de entrenamiento para Inceptionv3 (tamaño de entrada 299x299)
train_transforms_large = transforms.Compose([
    transforms.RandomResizedCrop(299),               # Recorte aleatorio y redimensionado a 299x299
    transforms.RandomHorizontalFlip(),               # Volteo horizontal aleatorio
    transforms.ToTensor(),                           # Convertir a tensor
    transforms.Normalize([0.485, 0.456, 0.406],      # Normalización (valores de ImageNet)
                         [0.229, 0.224, 0.225])
])

# Transformación de prueba para Inceptionv3 (299x299)
test_transforms_large = transforms.Compose([
    transforms.Resize(299),                          # Redimensionar la imagen a 299px
    transforms.CenterCrop(299),                      # Recorte centrado a 299x299
    transforms.ToTensor(),                           # Convertir a tensor
    transforms.Normalize([0.485, 0.456, 0.406],      # Normalización (valores de ImageNet)
                         [0.229, 0.224, 0.225])
])

# Cargar las imágenes de entrenamiento desde la carpeta "train"
train_dir = os.path.join(data_dir, 'train')  # Ruta de las imágenes de entrenamiento
test_dir = os.path.join(data_dir, 'test')    # Ruta de las imágenes de prueba

# Cargar el dataset con las transformaciones correspondientes (para VGG16 y ResNet50)
train_dataset_small = datasets.ImageFolder(train_dir, transform=train_transforms_small)

# Cargar el dataset para Inceptionv3 (transformaciones con 299x299)
train_dataset_large = datasets.ImageFolder(train_dir, transform=train_transforms_large)

# Dividir el dataset de entrenamiento en subconjuntos de entrenamiento (75%) y validación (25%)
train_size = int(0.75 * len(train_dataset_small))  # 75% de los datos para entrenamiento
val_size = len(train_dataset_small) - train_size   # 25% para validación

# Generar índices aleatorios para dividir el conjunto de datos
train_indices, val_indices = train_test_split(range(len(train_dataset_small)), test_size=val_size, random_state=42)

# Crear subconjuntos de entrenamiento y validación (para VGG16 y ResNet50)
train_subset_small = Subset(train_dataset_small, train_indices)  # Subconjunto de entrenamiento
val_subset_small = Subset(train_dataset_small, val_indices)      # Subconjunto de validación

# Crear subconjuntos de entrenamiento y validación (para Inceptionv3)
train_subset_large = Subset(train_dataset_large, train_indices)  # Subconjunto de entrenamiento
val_subset_large = Subset(train_dataset_large, val_indices)      # Subconjunto de validación

# Crear DataLoaders para cargar los datos en lotes pequeños (batch size 8)
batch_size = 8  # Tamaño del lote

# DataLoaders para VGG16 y ResNet50
train_loader_small = DataLoader(train_subset_small, batch_size=batch_size, shuffle=True)  # Conjunto de entrenamiento
val_loader_small = DataLoader(val_subset_small, batch_size=batch_size, shuffle=False)     # Conjunto de validación

# DataLoaders para Inceptionv3
train_loader_large = DataLoader(train_subset_large, batch_size=batch_size, shuffle=True)  # Conjunto de entrenamiento
val_loader_large = DataLoader(val_subset_large, batch_size=batch_size, shuffle=False)     # Conjunto de validación

# Cargar los datasets de prueba para VGG16/ResNet50 y Inceptionv3
test_dataset_small = datasets.ImageFolder(test_dir, transform=test_transforms_small)      # Dataset de prueba (VGG16/ResNet50)
test_dataset_large = datasets.ImageFolder(test_dir, transform=test_transforms_large)      # Dataset de prueba (Inceptionv3)

# DataLoaders para los conjuntos de prueba
test_loader_small = DataLoader(test_dataset_small, batch_size=batch_size, shuffle=False)  # Lotes de datos de prueba (VGG16/ResNet50)
test_loader_large = DataLoader(test_dataset_large, batch_size=batch_size, shuffle=False)  # Lotes de datos de prueba (Inceptionv3)


In [ ]:
#PARTE 3
# DEFINIR LOS MODELOS PREENTRENADOS

# Cargar modelos preentrenados de VGG16, ResNet50 e InceptionV3
vgg16 = models.vgg16(pretrained=True)  # Carga el modelo VGG16 con pesos preentrenados
resnet50 = models.resnet50(pretrained=True)  # Carga el modelo ResNet50 con pesos preentrenados
inceptionv3 = models.inception_v3(pretrained=True)  # Carga el modelo InceptionV3 con pesos preentrenados

# Congelar capas iniciales para que no se actualicen durante el entrenamiento
for param in vgg16.features.parameters():
    param.requires_grad = False           # No actualizar parámetros de VGG16
for param in resnet50.parameters():
    param.requires_grad = False           # No actualizar parámetros de ResNet50
for param in inceptionv3.parameters():
    param.requires_grad = False           # No actualizar parámetros de InceptionV3

# FINE TUNNING - Ajuste fino: cambiar la capa de salida para clasificación binaria
vgg16.classifier[6] = nn.Linear(4096, 2)  # Cambiar la capa de salida de VGG16 para 2 clases (perro/gato)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 2)  # Cambiar la capa de salida de ResNet50 para 2 clases
inceptionv3.fc = nn.Linear(inceptionv3.fc.in_features, 2)  # Cambiar la capa de salida de InceptionV3 para 2 clases

# Mover los modelos a la GPU para aceleración en el entrenamiento
vgg16 = vgg16.to(device)                  # Mover VGG16 a GPU
resnet50 = resnet50.to(device)            # Mover ResNet50 a GPU
inceptionv3 = inceptionv3.to(device)      # Mover InceptionV3 a GPU


In [ ]:
#PARTE 4
# SELECCIÓN DE HIPERPARÁMETROS Y CONFIGURACIÓN DE ENTRENAMIENTO

# Variación de Hiperparámetros
learning_rate = 0.001                 # Tasa de aprendizaje para el optimizador
num_epochs = 2                        # Número de épocas para el entrenamiento

# Definir los optimizadores y la función de pérdida
criterion = nn.CrossEntropyLoss()     # Función de pérdida para clasificación múltiple (Cross Entropy Loss)

# Crear optimizadores Adam para cada modelo con la tasa de aprendizaje definida
optimizer_vgg16 = optim.Adam(vgg16.parameters(), lr=learning_rate)              # Optimizador para VGG16
optimizer_resnet50 = optim.Adam(resnet50.parameters(), lr=learning_rate)        # Optimizador para ResNet50
optimizer_inceptionv3 = optim.Adam(inceptionv3.parameters(), lr=learning_rate)  # Optimizador para InceptionV3


In [ ]:
#PARTE 5
# FUNCIONES DE ENTRENAMIENTO Y EVALUACIÓN

# Función de entrenamiento
def train(model, dataloader, criterion, optimizer, is_inception=False):
    model.train()           # Cambiar a modo de entrenamiento
    running_loss = 0.0      # Inicializar pérdida acumulada
    running_corrects = 0    # Inicializar conteo de aciertos

    # Iterar sobre el DataLoader
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Mover los datos a la GPU

        optimizer.zero_grad()  # Limpiar los gradientes acumulados

        # Forward pass
        if is_inception:  # Si se está utilizando el modelo InceptionV3

            # En InceptionV3, se obtienen dos salidas (la principal y la auxiliar)
            outputs, aux_outputs = model(inputs)      # Obtener las predicciones
            loss1 = criterion(outputs, labels)        # Calcular la pérdida principal
            loss2 = criterion(aux_outputs, labels)    # Calcular la pérdida auxiliar
            loss = loss1 + 0.4 * loss2                # Dar menor peso a la pérdida auxiliar
        else:
            outputs = model(inputs)            # Obtener las predicciones
            loss = criterion(outputs, labels)  # Calcular la pérdida

        _, preds = torch.max(outputs, 1)  # Obtener las predicciones con mayor probabilidad

        # Backward pass and optimization
        loss.backward()   # Calcular los gradientes
        optimizer.step()  # Actualizar los parámetros del modelo

        # Calcular pérdida y precisión acumuladas
        running_loss += loss.item() * inputs.size(0)  # Acumular pérdida
        running_corrects += torch.sum(preds == labels.data)  # Acumular aciertos

    # Calcular pérdida y precisión por época
    epoch_loss = running_loss / len(dataloader.dataset)              # Pérdida media por época
    epoch_acc = running_corrects.double() / len(dataloader.dataset)  # Precisión media por época
    return epoch_loss, epoch_acc                                      # Retornar pérdida y precisión

# Función de evaluación
def evaluate(model, loader, criterion):
    model.eval()              # Cambiar a modo de evaluación
    running_loss = 0.0        # Inicializar pérdida acumulada
    correct = 0               # Inicializar conteo de aciertos
    total = 0                 # Inicializar conteo total de imágenes

    with torch.no_grad():  # Desactivar cálculo de gradientes
        # Iterar sobre el DataLoader
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # Mover los datos a la GPU
            outputs = model(images)                                # Obtener las predicciones
            loss = criterion(outputs, labels)                      # Calcular la pérdida

            running_loss += loss.item()                            # Acumular pérdida
            _, predicted = torch.max(outputs, 1)                   # Obtener las predicciones con mayor probabilidad
            total += labels.size(0)                                # Actualizar el conteo total
            correct += (predicted == labels).sum().item()          # Acumular aciertos

    accuracy = 100 * correct / total             # Calcular la precisión en porcentaje
    return running_loss / len(loader), accuracy  # Retornar pérdida y precisión


In [ ]:
#PARTE 6
# ENTRENAMIENTO DE LOS MODELOS

# Almacenar historial de precisión y pérdida
history = {'vgg16': [], 'resnet50': [], 'inceptionv3': []}  # Diccionario para guardar resultados de cada modelo

# Entrenar y evaluar los modelos
for epoch in range(num_epochs):             # Iterar sobre el número de épocas
    print(f"Epoch {epoch+1}/{num_epochs}")  # Imprimir número de época actual
    print("-" * 10)                         # Separador visual

    # VGG16
    train_loss, train_acc = train(vgg16, train_loader_small, criterion, optimizer_vgg16)  # Entrenar el modelo VGG16
    val_loss, val_acc = evaluate(vgg16, val_loader_small, criterion)                      # Evaluar el modelo VGG16
    history['vgg16'].append((train_loss, train_acc, val_loss, val_acc))                   # Almacenar resultados
    print(f"VGG16 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}")  # Imprimir resultados

    # ResNet50
    train_loss, train_acc = train(resnet50, train_loader_small, criterion, optimizer_resnet50)  # Entrenar el modelo ResNet50
    val_loss, val_acc = evaluate(resnet50, val_loader_small, criterion)                         # Evaluar el modelo ResNet50
    history['resnet50'].append((train_loss, train_acc, val_loss, val_acc))                      # Almacenar resultados
    print(f"ResNet50 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}")  # Imprimir resultados

    # Inceptionv3
    train_loss, train_acc = train(inceptionv3, train_loader_large, criterion, optimizer_inceptionv3, is_inception=True)  # Entrenar el modelo InceptionV3
    val_loss, val_acc = evaluate(inceptionv3, val_loader_large, criterion)                      # Evaluar el modelo InceptionV3
    history['inceptionv3'].append((train_loss, train_acc, val_loss, val_acc))                   # Almacenar resultados
    print(f"Inceptionv3 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}")  # Imprimir resultados


In [ ]:
#PARTE 7
# PREDICCIÓN EN EL CONJUNTO DE PRUEBA

# Función para realizar predicciones
def predict(model, dataloader):   # Definición de la función de predicción
    model.eval()                  # Configurar el modelo en modo de evaluación
    all_preds = []                # Lista para almacenar todas las predicciones
    with torch.no_grad():         # Desactivar el cálculo del gradiente
        for images, _ in dataloader:          # Iterar sobre los datos del dataloader
            images = images.to(device)        # Mover imágenes a la GPU
            outputs = model(images)           # Realizar la predicción
            _, preds = torch.max(outputs, 1)  # Obtener las predicciones (índices de clase)
            all_preds.extend(preds.cpu().numpy())  # Almacenar las predicciones en la lista
    return all_preds  # Retornar todas las predicciones

# Obtener predicciones
preds_vgg16 = predict(vgg16, test_loader_small)              # Predicciones del modelo VGG16 en el conjunto de prueba
preds_resnet50 = predict(resnet50, test_loader_small)        # Predicciones del modelo ResNet50 en el conjunto de prueba
preds_inceptionv3 = predict(inceptionv3, test_loader_large)  # Predicciones del modelo InceptionV3 en el conjunto de prueba


In [ ]:
#PARTE 8
# GENERAR INFORMES DE CLASIFICACIÓN

# Función para imprimir el informe de clasificación
def print_classification_report(y_true, y_pred):  # Definición de la función que toma las etiquetas verdaderas y predicciones
    print(classification_report(y_true, y_pred, target_names=test_dataset_small.classes))  # Imprimir el informe de clasificación con nombres de clases

# Cálculo de la precisión en el conjunto de prueba
test_labels = test_dataset_small.targets               # Obtener etiquetas verdaderas del conjunto de prueba
print("VGG16 Classification Report:")                  # Imprimir encabezado para el informe de VGG16
print_classification_report(test_labels, preds_vgg16)  # Generar y mostrar el informe para VGG16

print("ResNet50 Classification Report:")                  # Imprimir encabezado para el informe de ResNet50
print_classification_report(test_labels, preds_resnet50)  # Generar y mostrar el informe para ResNet50

print("InceptionV3 Classification Report:")                  # Imprimir encabezado para el informe de InceptionV3
print_classification_report(test_labels, preds_inceptionv3)  # Generar y mostrar el informe para InceptionV3


In [ ]:
#PARTE 9
# VISUALIZACIÓN DE RESULTADOS DE PERDIDA Y PRECISIÓN

# Función para graficar la historia de entrenamiento
def plot_history(history):                       # Definición de la función que toma el historial de métricas
    for model_name, metrics in history.items():  # Iterar sobre los modelos y sus métricas
        train_losses, train_accs, val_losses, val_accs = zip(*metrics)  # Desempaquetar las métricas en listas separadas
        plt.figure(figsize=(12, 5))              # Crear una nueva figura con tamaño específico

        # Pérdida
        plt.subplot(1, 2, 1)                           # Crear un subplot para la pérdida
        plt.plot(train_losses, label='Training Loss')  # Graficar la pérdida de entrenamiento
        plt.plot(val_losses, label='Validation Loss')  # Graficar la pérdida de validación
        plt.title(f'{model_name} Loss')                # Título del gráfico de pérdida
        plt.xlabel('Epoch')                            # Etiqueta del eje x
        plt.ylabel('Loss')                             # Etiqueta del eje y
        plt.legend()                                   # Mostrar la leyenda

        # Precisión
        plt.subplot(1, 2, 2)                             # Crear un subplot para la precisión
        plt.plot(train_accs, label='Training Accuracy')  # Graficar la precisión de entrenamiento
        plt.plot(val_accs, label='Validation Accuracy')  # Graficar la precisión de validación
        plt.title(f'{model_name} Accuracy')              # Título del gráfico de precisión
        plt.xlabel('Epoch')                              # Etiqueta del eje x
        plt.ylabel('Accuracy')                           # Etiqueta del eje y
        plt.legend()                                     # Mostrar la leyenda

        plt.tight_layout()  # Ajustar el diseño para que no se superpongan
        plt.show()          # Mostrar el gráfico

plot_history(history)  # Llamar a la función para graficar el historial de entrenamiento
